In [1]:
import numpy as np
import pandas as pd

from scipy.stats import mode
from scipy.stats import skew
from scipy.stats import kurtosis

datapath = 'Training Data/Session01/'

t_df = pd.read_csv(datapath + 'time.txt', sep = '\s+', header = None)

arm = pd.read_csv(datapath + 'armIMU.txt',sep = '\s+', header = None)
wrist = pd.read_csv(datapath + 'wristIMU.txt', sep = '\s+',header = None)

detected = pd.read_csv(datapath + 'detection.txt', sep = '\s+',header = None)

stride = 25


def extractFeatures(df):
    l = 0
    r = 3
    f = []
    for i in range(0,4):
        temp = df.iloc[:,l:r]
        f1 = list(temp.mean())
        
        C = temp.cov()
        f2 = []
        f2 += list(C.iloc[0,1:3])
        f2 += list(C.iloc[1,[0,2]])
        f2 += list(C.iloc[2,0:2])
        
        f3 = list(temp.apply(skew, axis = 0))
        f4 = list(temp.apply(kurtosis, axis = 0))
        
        f5 = list(temp.max())
        f6 = list(temp.min())
        
        l += 3
        r += 3
        ff = f1 + f2 + f3 + f4 + f5 + f6
        f += ff
    return f
    
extracted = []
y = []
X = pd.concat([arm, wrist], axis = 1)

for i in range(150, len(X), stride):
    extracted.append(extractFeatures(X.iloc[i-150:i,])) 
    label = mode(detected.iloc[i-150:i,])[0]
    y.append(label[0][0])

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



X = pd.DataFrame(extracted)
print(X.shape)

x_train, x_test, y_train, y_test = train_test_split(X, prediction, test_size=0.33)
randFor = RandomForestClassifier(class_weight='balanced', n_estimators = 10, max_depth = 10, criterion='entropy')
# Training the model
randFor.fit(x_train, y_train) 

# Make predictions on val data

y_pred = randFor.predict(x_test)
accuracy_score(y_pred, y_test)

(10173, 84)


0.958904109589041

In [ ]:
def createCSV(y):
    for int